<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/fine_tune_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
nn = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
#nn = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
!pip install transformers
!pip install datasets
cats_full = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics',
        'clinical_knowledge', 'college_biology', 'college_chemistry',
        'college_computer_science', 'college_mathematics', 'college_medicine',
        'college_physics', 'computer_security', 'conceptual_physics', 'econometrics',
        'electrical_engineering', 'elementary_mathematics', 'formal_logic',
        'global_facts', 'high_school_biology', 'high_school_chemistry',
        'high_school_computer_science', 'high_school_european_history',
        'high_school_geography', 'high_school_government_and_politics',
        'high_school_macroeconomics', 'high_school_mathematics',
        'high_school_microeconomics', 'high_school_physics',
        'high_school_psychology', 'high_school_statistics',
        'high_school_us_history', 'high_school_world_history',
        'human_aging', 'human_sexuality', 'international_law',
        'jurisprudence', 'logical_fallacies', 'machine_learning',
        'management', 'marketing', 'medical_genetics', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy',
        'prehistory', 'professional_accounting', 'professional_law',
        'professional_medicine', 'professional_psychology', 'public_relations',
        'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

cats = ['abstract_algebra', 'elementary_mathematics', 'formal_logic', 'global_facts',
        'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics',
        'high_school_psychology', 'human_aging', 'human_sexuality',
        'logical_fallacies', 'machine_learning', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'philosophy', 'professional_psychology', 'sociology', 'world_religions']

In [ ]:
from datasets import load_dataset

d = {}
for i in range(len(cats)):
  d[cats[i]] = load_dataset('cais/mmlu', cats[i])
#d.filter(lambda x: x["subject"] in cats)

In [10]:
d[cats[0]]

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 11
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 5
    })
})

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(nn)


def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

#tokenized_datasets = ds.map(tokenize_function, batched=True) ???

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(nn, num_labels=4, torch_dtype="auto")
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")
#training_args = TrainingArguments(output_dir="test_trainer")
metric = evaluate.load("accuracy")

In [ ]:
for i in d:
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset= i["test"],
      eval_dataset=i["validation"],
      compute_metrics=compute_metrics)
  trainer.train()